In [1]:
import pandas as pd
import numpy as np
import os
import io
import tensorflow as tf
from tensorflow.python.client import device_lib

In [2]:
NetBIOS = pd.read_csv('./01-12/DrDoS_NetBIOS.csv', low_memory=False, nrows=500000)
DNS = pd.read_csv('./01-12/DrDoS_DNS.csv', low_memory=False, nrows=500000)
LDAP = pd.read_csv('./01-12/DrDoS_LDAP.csv', low_memory=False, nrows=500000)
Portmap =  pd.read_csv('./03-11/Portmap.csv', low_memory=False, nrows=500000)

In [3]:
SNMP = pd.read_csv('./01-12/DrDoS_SNMP.csv', low_memory=False, nrows=500000)
SSDP = pd.read_csv('./01-12/DrDoS_SSDP.csv', low_memory=False, nrows=500000)
UDP = pd.read_csv('./01-12/DrDoS_UDP.csv', low_memory=False, nrows=500000)
SYN = pd.read_csv('./01-12/Syn.csv', low_memory=False, nrows=500000)

In [4]:
MSSQL = pd.read_csv('./01-12/DrDoS_MSSQL.csv', low_memory=False, nrows=500000)
NTP = pd.read_csv('./01-12/DrDoS_NTP.csv', low_memory=False, nrows=500000)
TFTP = pd.read_csv('./01-12/TFTP.csv', low_memory=False, nrows=500000)
UDPLag = pd.read_csv('./01-12/UDPLag.csv', low_memory=False, nrows=500000)

In [5]:
reflective_atacks = pd.concat([NetBIOS, DNS, LDAP, Portmap, MSSQL, NTP, SNMP, SSDP, UDP, SYN, TFTP, UDPLag])

In [6]:
# Rename the types of attacks
old_labels = ['DrDoS_SSDP', 'DrDoS_LDAP', 'DrDoS_SNMP', 'DrDoS_NetBIOS', 'DrDoS_MSSQL', 'DrDoS_UDP', 'DrDoS_DNS', 'DrDoS_NTP']
new_labels = ['SSDP', 'LDAP', 'SNMP', 'NetBIOS', 'MSSQL', 'UDP', 'DNS', 'NTP']
reflective_atacks[' Label'].replace(old_labels, new_labels, inplace=True)

In [7]:
# Drop NaN values
print('There are {} columns and {} rows in the Layer 1 Data.'.format(len(reflective_atacks.columns), len(reflective_atacks)))
reflective_atacks.dropna(axis='index', inplace=True)
reflective_atacks = reflective_atacks[~reflective_atacks.isin([np.nan, np.inf, -np.inf]).any(1)]
print('After dropping NaN values, there are {} columns and {} rows in Layer 1 Data.\n\n'.format(len(reflective_atacks.columns), len(reflective_atacks)))

There are 88 columns and 5562299 rows in the Layer 1 Data.
After dropping NaN values, there are 88 columns and 5362052 rows in Layer 1 Data.




In [8]:
# Remove overfitting columns (the same ones used in the original paper)
bad_columns = ['Unnamed: 0', ' Source IP', ' Destination IP', ' Source Port', ' Destination Port', ' Timestamp', 'SimillarHTTP', 'Flow ID']
reflective_atacks.drop(labels=bad_columns, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)


In [9]:
print('After dropping overfitting columns: \n\t there are {} columns and {} rows'.format(len(reflective_atacks.columns), len(reflective_atacks)))

After dropping overfitting columns: 
	 there are 80 columns and 5362052 rows


In [10]:
reflective_atacks[' Label'].value_counts()

SNMP       495288
UDP        494790
NetBIOS    489519
SSDP       489272
LDAP       487399
TFTP       485474
NTP        484073
MSSQL      483150
DNS        476722
Syn        440682
UDP-lag    330079
Portmap    177197
BENIGN      27968
WebDDoS       439
Name:  Label, dtype: int64

In [11]:
# One hot encoding
reflective_atacks = pd.concat([reflective_atacks,pd.get_dummies(reflective_atacks[' Label'])],axis=1)
reflective_atacks.drop([' Label'], axis=1, inplace=True)

In [12]:
reflective_atacks.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,BENIGN,DNS,LDAP,MSSQL,NTP,NetBIOS,Portmap,SNMP,SSDP,Syn,TFTP,UDP,UDP-lag,WebDDoS
0,17,1,2,0,884.0,0.0,442.0,442.0,442.0,0.0,0.0,0.0,0.0,0.0,8.840000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,442.0,442.0,442.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,663.0,442.0,0.0,-2,0,0,0,0,0,0,2,884,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,17,1,2,0,2346.0,0.0,1173.0,1173.0,1173.0,0.0,0.0,0.0,0.0,0.0,2.346000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,1173.0,1173.0,1173.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,1759.5,1173.0,0.0,-2,0,0,0,0,0,0,2,2346,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,17,1,2,0,942.0,0.0,471.0,471.0,471.0,0.0,0.0,0.0,0.0,0.0,9.420000e+08,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,471.0,471.0,471.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,706.5,471.0,0.0,-2,0,0,0,0,0,0,2,942,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,17,1,2,0,1020.0,0.0,510.0,510.0,510.0,0.0,0.0,0.0,0.0,0.0,1.020000e+09,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,510.0,510.0,510.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,765.0,510.0,0.0,-2,0,0,0,0,0,0,2,1020,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,17,2,2,0,1002.0,0.0,501.0,501.0,501.0,0.0,0.0,0.0,0.0,0.0,5.010000e+08,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,501.0,501.0,501.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,751.5,501.0,0.0,-2,0,0,0,0,0,0,2,1002,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [13]:
# Feature Selection
from sklearn.preprocessing import MinMaxScaler
# Import chi2 function 
from sklearn.feature_selection import chi2
label_start = 80

In [14]:
# Split into features (X) and target (Y)
reflective_X = reflective_atacks.iloc[:, 0:label_start]
reflective_Y = reflective_atacks.iloc[:, label_start:]

In [15]:
#reflective_X['SimillarHTTP'] = reflective_X['SimillarHTTP'].apply(pd.to_numeric, errors='coerce')
#reflective_X.dropna(axis='index', inplace=True)

In [16]:
#import numpy as np
#reflective_X = reflective_X[~reflective_X.isin([np.nan, np.inf, -np.inf]).any(1)]
#reflective_Y = reflective_Y[~reflective_Y.isin([np.nan, np.inf, -np.inf]).any(1)]

In [17]:
# Normalize all the values χ2 doesn't work with negative values
scaler = MinMaxScaler()
reflective_X = pd.DataFrame( 
    scaler.fit_transform(reflective_X), 
    columns=reflective_X.columns 
)

In [18]:
chi_scores = chi2(reflective_X , reflective_Y)

# Retrieve the P-Scores for each feature
p_vals = pd.Series(chi_scores[1],index = reflective_X.columns)

# Sort by lower to higher p-values 
p_vals.sort_values(ascending=True, inplace=True)

#print(p_vals)
#print("\t\t Layer 1 p-values \t\t\t\t\t\t Layer 2 p-values")
# Print all p-values in ascending order based on p-value, if they are >0.5, then mark them as red
#red='\033[31m'
#black='\033[30m'
#for i in range(len(p_vals)):
 #   str1 = "{}{:<38} = {}".format(black, p_vals.index[i], float(p_vals[i]))
  #  if float(p_vals[i]) > 0.5:
   #   str1 = "{}{:<38} = {}".format(red, p_vals.index[i], float(p_vals[i]))
   # print("{} \t {}| \t {}".format(str1, black))

In [19]:
pd.set_option('display.max_rows', None)
print(p_vals)

 Protocol                        0.000000e+00
Fwd PSH Flags                    0.000000e+00
 Bwd Packets/s                   0.000000e+00
 Min Packet Length               0.000000e+00
 Max Packet Length               0.000000e+00
 Packet Length Mean              0.000000e+00
 Packet Length Std               0.000000e+00
 RST Flag Count                  0.000000e+00
 ACK Flag Count                  0.000000e+00
 URG Flag Count                  0.000000e+00
 CWE Flag Count                  0.000000e+00
 Down/Up Ratio                   0.000000e+00
 Bwd IAT Min                     0.000000e+00
 Average Packet Size             0.000000e+00
 Avg Bwd Segment Size            0.000000e+00
Subflow Fwd Packets              0.000000e+00
 Subflow Fwd Bytes               0.000000e+00
Init_Win_bytes_forward           0.000000e+00
 Init_Win_bytes_backward         0.000000e+00
 act_data_pkt_fwd                0.000000e+00
 min_seg_size_forward            0.000000e+00
Idle Mean                        0